В данном задании нам нужно сделать из выборки (GSD_train.txt) библиотеку.

In [2]:
import nltk 

In [3]:
from nltk import sequential
import statistics
import pandas as pd
import numpy as np

In [4]:
def words_stats(sents, name = " "):

    words = [] 

    for sent in sents:
        for word_pos in sent:
            words.append(word_pos[0])

    words_lens = [len(word) for word in words] # Длины всех слов в корпусе
    mean_words = round(statistics.mean(words_lens), 2)
    less_than_4 = len([i for i in words_lens if i < 4])
    more_than_8 = len([j for j in words_lens if j > 8])

    print("Средняя длина слова в корпусе " + name + " = " + str(mean_words))
    print("Количество слов меньше 4-х символов в корпусе " + name + " = " + str(less_than_4) + " (" +
          str(round(less_than_4 / len(words)*100, 2)) + "%)")
    print("Количество слов больше 8-и символов в корпусе " + name + " = " + str(more_than_8)+ " (" +
          str(round(more_than_8/ len(words)*100, 2)) + "%)")


def go_thru_loop(train, test):
    for i in range(-5,-1):
        for j in range(4):
            if -i+j > 6:
                break
            else:
                print("Тренировка тэггера...")
                print("Длина суффикса ", i*-1)
                print("Минимальная стемма ", j)
                tagger = sequential.AffixTagger(train=train, affix_length=i,
                                                          min_stem_length=j, verbose=False)
                print("Результат:", round(tagger.evaluate(test), 3)*100, "%")
                print('-----------------------------')

def rmv_irr_tokens(sents):

    print('--------------------')
    print("Количество словоупотреблений до чистки: " + str(sum(len(i) for i in sents)))
    num_of_irr = 0

    for i in range(6):
        for sent in sents:
            for word_pos in sent:
                pos = list(word_pos)[1].split(',')[0]
                if pos in ['PNCT', 'NUMR', 'PRTF', 'PRCL', 'PREP', 'UNKN', 'CONJ', 'PRTS',
                                  'sing', 'plur', 'neut', '', 'masc', 'INTJ']:
                    sent.remove(word_pos)
                    num_of_irr += 1
                elif pos is 'ADJF' and len(pos) <= 4:
                    sent.remove(word_pos)
                    num_of_irr += 1
            if len(sent) == 0:
                sents.remove(sent)

    print("Количество удаленных токенов: " + str(num_of_irr))
    print("Количество словоупотреблений после чистки: " + str(sum(len(i) for i in sents)))
    print('--------------------')

def full_morph_to_pos(sents):

    sent_index = 0
    for sent in sents:
        word_pos_index = 0
        for word_pos in sent:
            word_pos = list(word_pos)
            sents[sent_index][word_pos_index] = list(sents[sent_index][word_pos_index])
            sents[sent_index][word_pos_index][1] = word_pos[1].split(',')[0]
            sents[sent_index][word_pos_index] = tuple(sents[sent_index][word_pos_index])
            word_pos_index += 1
        sent_index += 1


def apply_tagger_to_list(tagger, word_list, counter = 0):

    print("----------")
    for tok, tag in tagger.tag(word_list):
        if tag is None:
            counter += 1
        print("(%s, %s), " % (tok, tag))
    print("----------")
    print("Неопознанных слов: %d" % counter)
    print()

<>:49: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:49: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\smirn\AppData\Local\Temp\ipykernel_2340\3728504035.py:49: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif pos is 'ADJF' and len(pos) <= 4:


Направляем на файл и делаем из него словарь

In [5]:
data = r'GSD_train.txt'
df = pd.read_csv(data, delimiter = "\t", header=None) # Чтение csv
df

,0,1,2,3,4,5,6,7,8,9
0,1,Начальный,начальный,ADJ,JJL,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,2,amod,_,_
1,2,ролик,ролик,NOUN,NN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,17,nsubj,_,SpaceAfter=No
2,3,",",",",PUNCT,",",_,5,punct,_,_
3,4,или,или,CCONJ,CC,_,5,cc,_,_
4,5,опенинг,опенинг,NOUN,NN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,2,conj,_,_
...,...,...,...,...,...,...,...,...,...,...
96956,8,и,и,CCONJ,CC,_,9,cc,_,_
96957,9,разбит,разбить,VERB,VBNH,Animacy=Inan|Aspect=Perf|Case=Nom|Gender=Masc|...,6,conj,_,_
96958,10,колхозный,колхозный,ADJ,JJL,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,11,amod,_,_
96959,11,сад,сад,NOUN,NN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,9,nsubj:pass,_,SpaceAfter=No


In [6]:
my_tuple = df.drop(columns = [1, 4, 5, 6, 7, 8, 9], axis = 1)
my_tuple

,0,2,3
0,1,начальный,ADJ
1,2,ролик,NOUN
2,3,",",PUNCT
3,4,или,CCONJ
4,5,опенинг,NOUN
...,...,...,...
96956,8,и,CCONJ
96957,9,разбить,VERB
96958,10,колхозный,ADJ
96959,11,сад,NOUN


In [7]:
my_tuple_dict = my_tuple.to_dict()

In [8]:
ar = []
mega_ar = []
for i in my_tuple_dict[2]:
  if my_tuple_dict[2][i] == ',' or my_tuple_dict[2][i] == '.':
    mega_ar.append(ar)
    ar = []
    continue
  ar.append((my_tuple_dict[2][i], my_tuple_dict[3][i]))

Считаем количество единиц

In [9]:
len(mega_ar)

11306

In [10]:
size = int(len(mega_ar) * 0.9)
train_sents = mega_ar[:127980]
test_sents = mega_ar[127980:]

In [11]:
mega_ar = train_sents


rmv_irr_tokens(train_sents) # Удаляем все слова, кроме {'GRND', 'ADJS', 'VERB', 'PRED', 'INFN', 'COMP', 'ADVB', 'ADJF', 'NOUN'}
rmv_irr_tokens(test_sents)

words_stats(train_sents, name = "sents_train") # Длины слов после чистки
print('--------------------------')
words_stats(train_sents, name = "sents_train")

print("|||||||||||||||||||||||||")
go_thru_loop(train_sents, train_sents) # После удаления нерелевантных слов и знаков (результаты улучшились)

--------------------
Количество словоупотреблений до чистки: 85655
Количество удаленных токенов: 0
Количество словоупотреблений после чистки: 85655
--------------------
--------------------
Количество словоупотреблений до чистки: 0
Количество удаленных токенов: 0
Количество словоупотреблений после чистки: 0
--------------------
Средняя длина слова в корпусе sents_train = 5.69
Количество слов меньше 4-х символов в корпусе sents_train = 27700 (32.34%)
Количество слов больше 8-и символов в корпусе sents_train = 19254 (22.48%)
--------------------------
Средняя длина слова в корпусе sents_train = 5.69
Количество слов меньше 4-х символов в корпусе sents_train = 27700 (32.34%)
Количество слов больше 8-и символов в корпусе sents_train = 19254 (22.48%)
|||||||||||||||||||||||||
Тренировка тэггера...
Длина суффикса  5
Минимальная стемма  0


C:\Users\smirn\AppData\Local\Temp\ipykernel_2340\3728504035.py:32: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print("Результат:", round(tagger.evaluate(test), 3)*100, "%")


Результат: 57.199999999999996 %
-----------------------------
Тренировка тэггера...
Длина суффикса  5
Минимальная стемма  1
Результат: 47.8 %
-----------------------------
Тренировка тэггера...
Длина суффикса  4
Минимальная стемма  0
Результат: 65.3 %
-----------------------------
Тренировка тэггера...
Длина суффикса  4
Минимальная стемма  1
Результат: 56.3 %
-----------------------------
Тренировка тэггера...
Длина суффикса  4
Минимальная стемма  2
Результат: 47.099999999999994 %
-----------------------------
Тренировка тэггера...
Длина суффикса  3
Минимальная стемма  0
Результат: 69.3 %
-----------------------------
Тренировка тэггера...
Длина суффикса  3
Минимальная стемма  1
Результат: 62.1 %
-----------------------------
Тренировка тэггера...
Длина суффикса  3
Минимальная стемма  2
Результат: 54.1 %
-----------------------------
Тренировка тэггера...
Длина суффикса  3
Минимальная стемма  3
Результат: 45.7 %
-----------------------------
Тренировка тэггера...
Длина суффикса  2
Мини

In [12]:
rmv_irr_tokens(train_sents)


full_morph_to_pos(train_sents) #Теперь тренируем POS тэггер


pos_list = []
for sent in train_sents:
    for word_pos in sent:
        pos = word_pos[1]
        pos_list.append(pos)
print('--------------------')
print("Оставшиеся метки: ", set(pos_list))
print('--------------------')

go_thru_loop(train_sents, train_sents)

--------------------
Количество словоупотреблений до чистки: 85655
Количество удаленных токенов: 0
Количество словоупотреблений после чистки: 85655
--------------------
--------------------
Оставшиеся метки:  {'VERB', 'NOUN', 'SCONJ', 'AUX', 'CCONJ', 'X', 'ADJ', 'ADP', 'PRON', 'ADV', 'DET', 'PROPN', 'NUM', 'PUNCT', 'PART', 'SYM'}
--------------------
Тренировка тэггера...
Длина суффикса  5
Минимальная стемма  0


C:\Users\smirn\AppData\Local\Temp\ipykernel_2340\3728504035.py:32: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print("Результат:", round(tagger.evaluate(test), 3)*100, "%")


Результат: 57.199999999999996 %
-----------------------------
Тренировка тэггера...
Длина суффикса  5
Минимальная стемма  1
Результат: 47.8 %
-----------------------------
Тренировка тэггера...
Длина суффикса  4
Минимальная стемма  0
Результат: 65.3 %
-----------------------------
Тренировка тэггера...
Длина суффикса  4
Минимальная стемма  1
Результат: 56.3 %
-----------------------------
Тренировка тэггера...
Длина суффикса  4
Минимальная стемма  2
Результат: 47.099999999999994 %
-----------------------------
Тренировка тэггера...
Длина суффикса  3
Минимальная стемма  0
Результат: 69.3 %
-----------------------------
Тренировка тэггера...
Длина суффикса  3
Минимальная стемма  1
Результат: 62.1 %
-----------------------------
Тренировка тэггера...
Длина суффикса  3
Минимальная стемма  2
Результат: 54.1 %
-----------------------------
Тренировка тэггера...
Длина суффикса  3
Минимальная стемма  3
Результат: 45.7 %
-----------------------------
Тренировка тэггера...
Длина суффикса  2
Мини

In [13]:
cool_list  = ["фонетика", "теггер", "глокая", "куздра", "краб", "радовался", "хороший", "этот", "умереть", "быстро"]

test_sents = train_sents


rmv_irr_tokens(train_sents)
test_full_tagger = sequential.AffixTagger(train=train_sents, affix_length=-3, min_stem_length=0, verbose=False)
apply_tagger_to_list(test_full_tagger, cool_list)


full_morph_to_pos(train_sents)
test_pos_tagger = sequential.AffixTagger(train=train_sents, affix_length=-3, min_stem_length=0, verbose=False)
apply_tagger_to_list(test_pos_tagger, cool_list)

--------------------
Количество словоупотреблений до чистки: 85655
Количество удаленных токенов: 0
Количество словоупотреблений после чистки: 85655
--------------------
----------
(фонетика, NOUN), 
(теггер, PROPN), 
(глокая, PROPN), 
(куздра, NOUN), 
(краб, NOUN), 
(радовался, None), 
(хороший, ADJ), 
(этот, DET), 
(умереть, VERB), 
(быстро, ADV), 
----------
Неопознанных слов: 1

----------
(фонетика, NOUN), 
(теггер, PROPN), 
(глокая, PROPN), 
(куздра, NOUN), 
(краб, NOUN), 
(радовался, None), 
(хороший, ADJ), 
(этот, DET), 
(умереть, VERB), 
(быстро, ADV), 
----------
Неопознанных слов: 1



Делаем n-граммные модели

In [14]:
unigram_tagger = nltk.UnigramTagger(train_sents)
bigram_tagger = nltk.BigramTagger(train_sents)
trigram_tagger = nltk.TrigramTagger(train_sents)

In [15]:
print('Unigram:\t', unigram_tagger.accuracy(train_sents),
      '\nBigram:\t\t', bigram_tagger.accuracy(train_sents),
      '\nTrigram:\t', trigram_tagger.accuracy(train_sents))

Unigram:	 0.9840756523261923 
Bigram:		 0.9566166598564007 
Trigram:	 0.9562897670889031


СОздаём файл, куда записываем точное количество n-граммных моделей

In [24]:
f = open('N-gramm.txt', 'w+')
f.write(str(unigram_tagger))
f.write(str(bigram_tagger))
f.write(str(trigram_tagger))
f.close()